In [1]:
import yfinance as yf
import pandas as pd


In [2]:
nasdaq_ticker = "^IXIC"
nse_ticker = "^NSEI"

In [3]:
nasdaq_data = yf.download(nasdaq_ticker, start="2010-01-01", end="2023-5-1")


[*********************100%***********************]  1 of 1 completed


In [4]:
nse_data = yf.download(nse_ticker, start="2010-01-01", end="2023-5-1")


[*********************100%***********************]  1 of 1 completed


In [5]:
nasdaq_close = nasdaq_data['Close']
nse_close = nse_data['Close']

correlation_coefficient = nasdaq_close.corr(nse_close)

print("Correlation Coefficient: ", correlation_coefficient)

Correlation Coefficient:  0.9513138758576782


In [6]:
nasdaq_returns = nasdaq_close.pct_change()
nse_returns = nse_close.pct_change()


In [7]:
lag = range(1, 21)

In [8]:
correlations = []
for l in lag:
    nasdaq_lagged_returns = nasdaq_returns.shift(l)
    correlation = nasdaq_lagged_returns.corr(nse_returns)
    correlations.append(correlation)

In [9]:
optimal = lag[correlations.index(max(correlations))]


In [10]:
leadlag = "NASDAQ leads" if correlations.index(max(correlations)) > 0 else "NASDAQ lags"

In [27]:
print("Lead-Lag Relationship: ", leadlag)


Lead-Lag Relationship:  NASDAQ lags


In [12]:
def keltner_channel(data, n=20, k=2):
    typical_price = (data['High'] + data['Low'] + data['Close']) / 3
    sma = typical_price.rolling(n).mean()
    atr = data['High'] - data['Low']
    upper = sma + (k * atr)
    lower = sma - (k * atr)
    return upper, lower

In [13]:
def bollinger_bands(data, n=20, k=2):
    sma = data['Close'].rolling(n).mean()
    std = data['Close'].rolling(n).std()
    upper = sma + (k * std)
    lower = sma - (k * std)
    return upper, lower

def macd_(data, n_fast=12, n_slow=26):
    ema_fast = data['Close'].ewm(span=n_fast).mean()
    ema_slow = data['Close'].ewm(span=n_slow).mean()
    macd_line = ema_fast - ema_slow
    signal_line = macd_line.ewm(span=9).mean()
    return macd_line, signal_line

In [15]:
keltner_upper1, keltner_lower1 = keltner_channel(nasdaq_data)
keltner_upper2, keltner_lower2 = keltner_channel(nse_data)

bollinger_upper1, bollinger_lower1 = bollinger_bands(nasdaq_data)
bollinger_upper2, bollinger_lower2 = bollinger_bands(nse_data)


macd_line1, signal_line1 = macd_(nasdaq_data)
macd_line2, signal_line2 = macd_(nse_data)


print("Keltner Channel (Upper Band):")
print(keltner_upper1)
print("\nKeltner Channel (Lower Band):")
print(keltner_lower1)
print("\nBollinger Bands (Upper Band):")
print(bollinger_upper1)
print("\nBollinger Bands (Lower Band):")
print(bollinger_lower1)
print("\nMACD Line:")
print(macd_line1)
print("\nSignal Line:")
print(signal_line1)

print("Keltner Channel (Upper Band):")
print(keltner_upper2)
print("\nKeltner Channel (Lower Band):")
print(keltner_lower2)
print("\nBollinger Bands (Upper Band):")
print(bollinger_upper2)
print("\nBollinger Bands (Lower Band):")
print(bollinger_lower2)
print("\nMACD Line:")
print(macd_line2)
print("\nSignal Line:")
print(signal_line2)

Keltner Channel (Upper Band):
Date
2010-01-04             NaN
2010-01-05             NaN
2010-01-06             NaN
2010-01-07             NaN
2010-01-08             NaN
                  ...     
2023-04-24    12326.428174
2023-04-25    12426.203320
2023-04-26    12321.854639
2023-04-27    12467.460824
2023-04-28    12360.333968
Length: 3353, dtype: float64

Keltner Channel (Lower Band):
Date
2010-01-04             NaN
2010-01-05             NaN
2010-01-06             NaN
2010-01-07             NaN
2010-01-08             NaN
                  ...     
2023-04-24    11753.307080
2023-04-25    11659.441602
2023-04-26    11782.174951
2023-04-27    11655.101449
2023-04-28    11779.736312
Length: 3353, dtype: float64

Bollinger Bands (Upper Band):
Date
2010-01-04             NaN
2010-01-05             NaN
2010-01-06             NaN
2010-01-07             NaN
2010-01-08             NaN
                  ...     
2023-04-24    12316.892530
2023-04-25    12311.900802
2023-04-26    12286.40313

In [16]:
keltner_params = [(10, 1.5), (20, 2.0), (30, 2.5)]
bollinger_params = [(20, 1.5), (30, 2.0), (40, 2.5)]
macd_params = [(12, 26), (15, 30), (20, 40)]

In [17]:
best_parameters = None
best_metric = float('-inf')

In [25]:
for keltner1, keltner2 in keltner_params:
    for bollinger1, bollinger2 in bollinger_params:
        for macd_fast, macd_slow in macd_params:
            keltner_upper, keltner_lower = keltner_channel(nasdaq_data, keltner1, keltner2)
            bollinger_upper, bollinger_lower = bollinger_bands(nasdaq_data, bollinger1, bollinger2)
            macd_line, signal_line = macd_(nasdaq_data, macd_fast, macd_slow)
           
            returns = nasdaq_data['Close'].pct_change()
            annualized_return = np.prod(returns + 1) ** (252 / len(returns)) - 1
            
            if annualized_return > best_metric:
                best_parameters = (keltner1, keltner2, bollinger1, bollinger2, macd_fast, macd_slow)
                best_metric = annualized_return

print("Best Parameters for NASDAQ:", best_parameters)
print("Best Metric FOR NASDAQ:", best_metric)

Best Parameters for NASDAQ: (10, 1.5, 20, 1.5, 12, 26)
Best Metric FOR NASDAQ: 0.13347676852201573


In [21]:
import numpy as np

In [28]:
keltner_n = 10
keltner_k = 1.5
bollinger_n = 20
bollinger_k = 1.5
macd_fast = 12
macd_slow = 26

In [30]:
nse_keltner_upper, nse_keltner_lower = keltner_channel(nse_data, keltner1, keltner2)
nse_bollinger_upper, nse_bollinger_lower = bollinger_bands(nse_data, bollinger1, bollinger2)
nse_macd_line, nse_signal_line = macd_(nse_data, macd_fast, macd_slow)




In [31]:
nse_signals = pd.DataFrame(index=nse_data.index)
nse_signals['Keltner'] = np.where(nse_data['Close'] > nse_keltner_upper, -1, np.where(nse_data['Close'] < nse_keltner_lower, 1, 0))
nse_signals['Bollinger'] = np.where(nse_data['Close'] > nse_bollinger_upper, -1, np.where(nse_data['Close'] < nse_bollinger_lower, 1, 0))
nse_signals['MACD'] = np.where(nse_macd_line > nse_signal_line, 1, np.where(nse_macd_line < nse_signal_line, -1, 0))

In [32]:
nse_signals['Date'] = nse_signals.index
nse_signals['Return'] = nse_data['Close'].pct_change()


In [33]:
print("\nNSE Signals:")
print(nse_signals)



NSE Signals:
            Keltner  Bollinger  MACD       Date    Return
Date                                                     
2010-01-04        0          0     0 2010-01-04       NaN
2010-01-05        0          0     1 2010-01-05  0.008734
2010-01-06        0          0     1 2010-01-06  0.000739
2010-01-07        0          0    -1 2010-01-07 -0.003540
2010-01-08        0          0    -1 2010-01-08 -0.003487
...             ...        ...   ...        ...       ...
2023-04-24       -1          0     1 2023-04-24  0.006772
2023-04-25       -1          0     1 2023-04-25  0.001457
2023-04-26       -1          0     1 2023-04-26  0.002496
2023-04-27       -1          0     1 2023-04-27  0.005695
2023-04-28       -1          0     1 2023-04-28  0.008370

[3268 rows x 5 columns]
